# A Quick Look at the MIMIC II Data Set

### First install some dependencies

These dependencies are for all the notebooks, so you don't need to run them in the others, although I've included this cell in each one.

In [ ]:
### you may need to install jinja2. To do so,
# run this cell
!python -m pip install jinja2
!python -m pip install pyarrow
!python -m pip install -U git+https://github.com/chapmanbe/dminteract#egg=dminteract

In [ ]:
import warnings
warnings.simplefilter("ignore")
from m5utils.mutils import *
from dminteract.modules.m5 import *


MIMIC II data are stored in a relational database. This is not an exploration of relational database theory or data modeling, but here is my novice quick description.

* Relational databases seek to achieve accurate data representation by eliminating (reducing)  data redundancies and thus the opportunities for data inconsistencies.

This is achieved by splitting data across **tables** and then **joining** the data back together when required.

### First we need to generate a connection to the MIMIC database

As you work through this notebook, you might occasionally get an error that looks something like this (although much longer):

```Python
---------------------------------------------------------------------------
BrokenPipeError                           Traceback (most recent call last)
...
(2006, "MySQL server has gone away (BrokenPipeError(32, 'Broken pipe'))")

```

This just means that the connection with the database has timed out. All you need to do is come back up here and rerun the code below to get a new database connection.

In [ ]:
conn = get_mimic_connection()

## Let's take a look at the tables

>Before you can do anything, you have to understand tables. If you don't have a table, you have nothing to work on. The table is the standard unit of information in a relational database. Everything revolves around tables. Tables are composed of rows and columns. And while that sounds simple, the sad truth is that tables are not simple. (*The Definitive Guide to SQLite*, p. 80 [owens2006definitive})


Since I said data are split across tables, let's look at the tables in the MIMIC II demo database.

### Take a look at the Tables in the Database

In [ ]:
tables = get_tables(conn)
HTML(dlist(tables["Tables_in_mimic2"].to_list(), ncols=5, sort=True))

## By my reckoning, we have three types of tables

* Database-internal table(s) (db_schema)
* Tables that start with `a_`
* Tables that start with `d_`
* Everything else

## What are in the tables?
### We can use the MySQL command `DESCRIBE` to see the definitions in each table

In [ ]:
table_names = [row["Tables_in_mimic2"] for _,row in tables.iterrows()]

table_descriptions = get_table_descriptons(conn)



### Take a look at `d_patients`

This is the table that defines the individuals in the rest of the database. Each individual is  characterized by a unique identifer (`subject_id`), their sex described by a single-character, a date of birth, a date of death, and a single-character flag indicating whether the patient died in the hospital. 

In [ ]:
display(table_descriptions["d_patients"])

view_table("d_patients", 5, conn)

### Examine `medevents`

For a patient being given medication (medication event), we would want to know things like who was the medicine given to, who gave it to them, what was the medicine, when was it given, etc.

Examining the `medevents` table we an see  the nature of a relational database

* Instead of the name of the medication, we have `itemid` which refers to a medication defined in another table (in this case `d_meditems`)
* Instead of the name of the person administering the medication, we have `cgid` (caregiver id) which refers to someone defined in the table `d_caregivers`.

In [ ]:
display(table_descriptions["medevents"])
view_table("medevents", 5, conn)

Similar to the `d_patients` table, there are values like dose that are provided explicity (e.g. 1350) and the unit of measurement for dose (`doseuom=Uhr` (unit per hour), but there are also a large number of values (recognized by the `id` component in the name) that are references to values defined in other tables. These ids are called **foreign keys.** Some of these include:

* `subect_id` that is defined in `d_patients`
* `cgid` that is defined `d_caregivers`
* `itemid` that is defined in `d_meditems`

Another important idea is represented in the column named `Null`. If the value is `YES`, that means the value is allowed to be missing. A value of `NO` means that a value must be provided. The issue of missing data will be very important for many applications.

To fully understand an entry, we need to join the values that are provided in a `medevent` entry with the values defined in the other tables referred to with the `id` values (foreign keys).

### Here is an example

* Joining information from `medevents`, `d_caregivers`, `d_careunits`, `d_meditems`, and `d_patients`
* I'm randomly selecting 50 cases

In [ ]:
devents = pd.read_sql(
"""SELECT me.subject_id,
          dp.sex,
          dp.dob,
          me.charttime,
          dm.label as medname,
          me.dose,
          me.doseuom,
          me.route,
          me.cgid,
          dc.label as cglabel,
          du.label as cuname
   FROM ((((medevents AS me 
              INNER JOIN d_caregivers AS dc ON 
                  me.cgid = dc.cgid)
              INNER JOIN d_careunits AS du ON
                  me.cuid = du.cuid) 
              INNER JOIN d_meditems as dm ON
                  me.itemid = dm.itemid) 
              INNER JOIN d_patients as dp ON
                  me.subject_id = dp.subject_id) ORDER BY RAND() LIMIT 50""",conn)
devents.head()

### With a little bit of manipulation we can get the age

* Compute age
* Rename some of the columns, drop others
* **Note:** I'm doing all this in Python. If I were better at SQL, these steps could all be done within the query

In [ ]:
devents["age in years"] = devents.apply(lambda r: (r.charttime - r.dob).days/365.2425, axis=1)
devents2 = devents[["subject_id", "sex", "age in years", "charttime", "medname", 
                   "dose", "doseuom", "route", "cgid", "cglabel", "cuname"]]
devents2

### Here is a simple (but ugly) view of what is in each table

In [ ]:
tcs = get_table_columns(table_descriptions)
HTML(ddict(tcs))

#### Here is a simple interactive view of foreign keys and what they (might) refer to

The displayed table shows for a selected column (`col`) from `tbl` all the other MIMIC2 tables that have that same column name.

In [ ]:

    
table_names = widgets.Dropdown(options = tcs.keys())
col_names = widgets.Dropdown(options = tcs[table_names.value])
lbl = widgets.HTML(value="Tables where is also found.")
def update_col_choices(change):
    col_names.options = tcs[change.new]
    
table_names.observe(update_col_choices, names="value")
def get_db_links(tbls, tbl, col):
    lbl.value = markdown("## Tables where `%s` is also found"%col)
    tables = [t for t in tbls if col in tbls[t] and t != tbl]
    return HTML(dlist(tables, ncols=8, sort=True))
display(lbl)
i1 = interact(get_db_links, tbls=fixed(tcs), tbl = table_names, col=col_names)

### Some general comments

The database we are using is a *demo* database. Some tables are empty and others have values and foreign keys that I cannot figure. For example, I cannot find where the column `solutionid` in `medevents` is referring to, so we cannot decipher. There is not great documentation.

With completion of some simple online ethics training, you can get access to the complete version of [MIMIC (now MIMIC 3)](https://mimic.physionet.org/), which is well documented, has lots of example code available, and is one of the most valuable resources for clinical informatics research in the world.

### Questions

In [ ]:
for q in question_banks["qbank1"].values():
    display(q)

### [Go on to the next notebook](visualize_case_timeline.ipynb)

In [ ]:
for t in table_names:
    print(t)
    display(view_table(t, 20, conn))
    print("-"*40)